<a href="https://colab.research.google.com/github/Kanakanajm/nnti/blob/main/NNTIProject/notebooks/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Language model inference

The goal if this first task is to familiarize yourself with the huggingface transformers and dataset libraries. You will learn how to load and tokenize a dataset, how to load a pre-trained language model, and finally, how to run a model in inference mode.

Your task is to complete the missing code blocks below.

In [2]:
# download the requirements.txt hosted on a Github public gist
!wget https://gist.githubusercontent.com/CamiloMartinezM/7eb6f86e2642e24776c90433c29133de/raw/8e7d8ab19bc8f55fe2e837c1d1d0fe3026519fb1/requirements.txt -O requirements.txt
# install the required libraries
!pip install -r requirements.txt

--2024-02-29 18:38:20--  https://gist.githubusercontent.com/CamiloMartinezM/7eb6f86e2642e24776c90433c29133de/raw/8e7d8ab19bc8f55fe2e837c1d1d0fe3026519fb1/requirements.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     125  --.-KB/s    in 0s      

2024-02-29 18:38:21 (6.52 MB/s) - ‘requirements.txt’ saved [125/125]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
# import dependencies
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
import tqdm
import json
import os

from datasets import (
    load_dataset,
    load_dataset_builder,
    get_dataset_split_names,
    get_dataset_config_names,
)
from transformers import (
    XGLMTokenizer,
    XGLMTokenizerFast,
    XGLMForCausalLM,
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig
)

# set up figure parameters to make them look nice
plt.rcParams["axes.formatter.use_mathtext"] = True
matplotlib.rcParams["font.family"] = "cmr10"
matplotlib.rcParams["axes.unicode_minus"] = False
matplotlib.rcParams.update({"font.size": 11})
matplotlib.rcParams['figure.dpi'] = 150

# other utils functions
from copy import deepcopy
from pprint import pprint, pformat

def left_indent(text, indent: str = "\t"):
    return "".join([indent + l for l in text.splitlines(True)])

def pprint_tab(obj, indent: str = "\t"):
    print(left_indent(pformat(obj), indent))

def get_available_objects(data):
    return [obj for obj in dir(data) if not obj.startswith("__")]

def copy(obj):
    return deepcopy(obj)

def save_tensor_to(dst: str, filename: str, t: torch.Tensor, overwrite: bool = True) -> None:
    if not os.path.exists(dst):
        os.makedirs(dst)
    fullpath = os.path.join(dst, filename)
    # save to file if it does not exist or if it does exist and overwrite = True
    if (os.path.exists(fullpath) and overwrite) or not os.path.exists(fullpath):
        torch.save(t, fullpath)

## Explore dataset

In [4]:
DATA_SET_NAME = "facebook/flores" # specify dataset name
MODEL_NAME = "facebook/xglm-564M" # specify model name
# MODEL_NAME = "gpt2" # specify model name

In [5]:
# Explore a dataset
LANGUAGE_CODE = "deu_Latn" # Language to explore

# covered language codes can be found here: https://github.com/openlanguagedata/flores?tab=readme-ov-file#language-coverage

ds_builder = load_dataset_builder(DATA_SET_NAME, LANGUAGE_CODE, trust_remote_code=True)
print(ds_builder.info.description) # print the dataset description

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The creation of FLORES-200 doubles the existing language coverage of FLORES-101. 
Given the nature of the new languages, which have less standardization and require 
more specialized professional translations, the verification process became more complex. 
This required modifications to the translation workflow. FLORES-200 has several languages 
which were not translated from English. Specifically, several languages were translated 
from Spanish, French, Russian and Modern Standard Arabic. Moreover, FLORES-200 also 
includes two script alternatives for four languages. FLORES-200 consists of translations 
from 842 distinct web articles, totaling 3001 sentences. These sentences are divided 
into three splits: dev, devtest, and test (hidden). On average, sentences are approximately 
21 words long.



In [6]:
# print the features (columns) of the dataset
pprint(ds_builder.info.features)

{'URL': Value(dtype='string', id=None),
 'domain': Value(dtype='string', id=None),
 'has_hyperlink': Value(dtype='int32', id=None),
 'has_image': Value(dtype='int32', id=None),
 'id': Value(dtype='int32', id=None),
 'sentence': Value(dtype='string', id=None),
 'topic': Value(dtype='string', id=None)}


In [7]:
# get the available splits
pprint(ds_builder.info.splits)

None


## Load data, tokenize, and batchify

In [8]:
# specify languages
LANGUAGES = [
    "eng_Latn",
    "spa_Latn",
    "ita_Latn",
    "deu_Latn",
    "arb_Arab",
    "tel_Telu",
    "tam_Taml",
    "quy_Latn"
]

In [9]:
# Set up the splits to download
USE_SPLITS = ["dev", "devtest"]

"""
load flores data for each language
structure:
dataset_per_lang = {
  language: {
      "dataset": {
           split (dev/devtest): {
               "raw": raw dataset (without tokenization),
               "tokenized": tokenized dataset
           }
      },
      "dataloader": {
           split (dev/devtest): dataloader for the split
      }
  }
}
"""
dataset_per_lang = {}
for language in LANGUAGES:
    print(f"Loading dataset for {language}", end="... ")

    # add a dataloader key set to None, they are defined in the cell tagged
    # @dataloader-creation
    dataset_per_lang[language] = {"dataset": {}, "dataloader": None}

    for split in USE_SPLITS:
        dataset_per_lang[language]["dataset"][split] = {}
        dataset_per_lang[language]["dataset"][split]["raw"] = load_dataset(
            DATA_SET_NAME,
            language,
            split=split,
            trust_remote_code=True,
            cache_dir="../cache/languages",
        )

    print("done")

Loading dataset for eng_Latn... 

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

done
Loading dataset for spa_Latn... 

Generating dev split: 0 examples [00:00, ? examples/s]

...

Generating devtest split: 0 examples [00:00, ? examples/s]

done
Loading dataset for quy_Latn... 

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

done


In [10]:
# let's look at the English subset
EX_DATASET_LANG = "eng_Latn"
english_dataset = dataset_per_lang[EX_DATASET_LANG]["dataset"]["dev"]["raw"]
print(f"Size of the english dataset: {english_dataset.info.dataset_size}")
print("Features:")
pprint_tab(english_dataset.info.features)
print("\nSplits:")
pprint_tab(english_dataset.info.splits)

Size of the english dataset: 501481
Features:
	{'URL': Value(dtype='string', id=None),
	 'domain': Value(dtype='string', id=None),
	 'has_hyperlink': Value(dtype='int32', id=None),
	 'has_image': Value(dtype='int32', id=None),
	 'id': Value(dtype='int32', id=None),
	 'sentence': Value(dtype='string', id=None),
	 'topic': Value(dtype='string', id=None)}

Splits:
	{'dev': SplitInfo(name='dev',
	                  num_bytes=245488,
	                  num_examples=997,
	                  shard_lengths=None,
	                  dataset_name='flores'),
	 'devtest': SplitInfo(name='devtest',
	                      num_bytes=255993,
	                      num_examples=1012,
	                      shard_lengths=None,
	                      dataset_name='flores')}


In [11]:
# let's look at an individual sample from the dataset
def get_sample(idx: int, lang: str, split: str, data: str):
    return dataset_per_lang[lang]['dataset'][split][data][idx]

print(f"Viewing raw samples from {EX_DATASET_LANG}:")
for split in USE_SPLITS:
    first_sample = get_sample(0, EX_DATASET_LANG, split, "raw")
    last_sample = get_sample(-1, EX_DATASET_LANG, split, "raw")
    dataset_len = len(dataset_per_lang[EX_DATASET_LANG]["dataset"][split]["raw"]) - 1

    print("")
    print(f"\tFirst sample from {split} split:")
    pprint_tab(first_sample, indent="\t\t")
    print("")
    print(f"\t{dataset_len}-th sample from {split} split:")
    pprint_tab(last_sample, indent="\t\t")

Viewing raw samples from eng_Latn:

	First sample from dev split:
		{'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
		 'domain': 'wikinews',
		 'has_hyperlink': 0,
		 'has_image': 0,
		 'id': 1,
		 'sentence': 'On Monday, scientists from the Stanford University School of '
		             'Medicine announced the invention of a new diagnostic tool that '
		             'can sort cells by type: a tiny printable chip that can be '
		             'manufactured using standard inkjet printers for possibly about '
		             'one U.S. cent each.',
		 'topic': 'health'}

	996-th sample from dev split:
		{'URL': 'https://en.wikivoyage.org/wiki/Funeral_travel',
		 'domain': 'wikivoyage',
		 'has_hyperlink': 0,
		 'has_image': 0,
		 'id': 997,
		 'sentence': 'In all cases, you must book by phone directly with the airline.',
		 'topic': 'Reason to travel/Funeral travel'}

	First sample from devtest split:
		{'URL': 'https

In [12]:
# global variable that defines if per-batch padding should be used, instead
# of padding over the whole dataset
PER_BATCH_PADDING = True

# tokenize the data

# load a pre-trained tokenizer from the huggingface hub
# if this throws an error and one is using a conda environment, one has to
# install sentencepiece library: conda install -c huggingface sentencepiece
try:
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME, cache_dir="../cache/tokenizers"
    )
except ValueError:
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME, cache_dir="../cache/tokenizers", use_fast=False
    )

# gpt2 does not have a padding token, so we have to add it manually
if MODEL_NAME == "gpt2":
    tokenizer.add_special_tokens({"pad_token": tokenizer.unk_token})


# Specify the tokenization function
# If padding is specified, then it is used inside the tokenizer function.
def tokenization(example, padding: str = None):
    return tokenizer(
        example["sentence"],
        padding=padding if padding else False,
        truncation=True if padding else False,
        return_tensors="pt",
    )

def add_batch_dimension(example):
    """
    Adds a batch dimension to the tensors.
    This function assumes the tensors are already in PyTorch tensors and simply
    unsqueezes them at the first dimension.
    See https://stackoverflow.com/questions/57237352/what-does-unsqueeze-do-in-pytorch
    """
    example["input_ids"] = example["input_ids"].unsqueeze(0)
    example["attention_mask"] = example["attention_mask"].unsqueeze(0)
    return example


for language in dataset_per_lang:
    for split in dataset_per_lang[language]["dataset"]:
        # If we are to pad the whole dataset, set `padding` to longest and pass it
        # to the tokenization function
        if not PER_BATCH_PADDING:
            raw_dataset = copy(dataset_per_lang[language]["dataset"][split]["raw"])

            # Tokenize the dataset
            tokenized_dataset = raw_dataset.map(
                lambda example: tokenization(example, padding="longest"), batched=True
            )

            # Update the tokenized dataset with Pytorch format
            tokenized_dataset.set_format(
                type="torch", columns=["input_ids", "attention_mask"]
            )

            # Apply unsqueeze operation
            tokenized_dataset = tokenized_dataset.map(
                lambda example: add_batch_dimension(example),
                batched=False,  # Set batched=False to apply function to each example individually
            )

            dataset_per_lang[language]["dataset"][split]["tokenized"] = tokenized_dataset
        else: # If per batch padding is to be used, we cannot tokenize the dataset here,
            dataset_per_lang[language]["dataset"][split]["tokenized"] = None

tokenizer_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

In [14]:
# construct a pytorch data loader for each dataset

# Dynamically pad to the longest sequence in the batch
def per_batch_padding_collate_fn(batch, padding: str = "longest"):
    sentences = [item['sentence'] for item in batch]
    batch_padded = tokenizer(
        sentences,
        padding=padding,
        truncation=True,
        return_tensors="pt",
    )
    return batch_padded

BATCH_SIZE = 2 # for testing purposes, we start with a batch size of 2. You can change this later.

for language in dataset_per_lang:
    dataset_per_lang[language]["dataloader"] = {}
    print(f"Creating dataloaders for {language} (", end="")
    for i, split in enumerate(dataset_per_lang[language]["dataset"]):
        # If `PER_BATCH_PADDING` is False, then the padding was applied earlier and value of
        # the key 'tokenized' is not None
        # If it is True however, then we pass a collate_fn to dynamically apply padding
        # per batch and initialize the DataLoader
        if not PER_BATCH_PADDING:
            dataset = dataset_per_lang[language]["dataset"][split]["tokenized"]
        else:
            dataset = dataset_per_lang[language]["dataset"][split]["raw"]

        # Set the BATCH_SIZE equal to the length of the dataset, if BATCH_SIZE == -1
        if BATCH_SIZE == -1:
            BATCH_SIZE = len(dataset)

        if not PER_BATCH_PADDING:
            dataset_per_lang[language]["dataloader"][split] = torch.utils.data.DataLoader(
                dataset,
                batch_size=BATCH_SIZE,
                shuffle=False
            )
        else:
            dataset_per_lang[language]["dataloader"][split] = torch.utils.data.DataLoader(
                dataset,
                batch_size=BATCH_SIZE,
                collate_fn=per_batch_padding_collate_fn,
                shuffle=True  # or False, depending on your needs
            )

        print(f"{split}: {len(dataset_per_lang[language]['dataloader'][split])}", end="")
        if i < len(dataset_per_lang[language]["dataset"]) - 1:
            print(", ", end="")

        if BATCH_SIZE == len(dataset):
            BATCH_SIZE = -1

    print(" instances)")

Creating dataloaders for eng_Latn (dev: 499, devtest: 506 instances)
Creating dataloaders for spa_Latn (dev: 499, devtest: 506 instances)
Creating dataloaders for ita_Latn (dev: 499, devtest: 506 instances)
Creating dataloaders for deu_Latn (dev: 499, devtest: 506 instances)
Creating dataloaders for arb_Arab (dev: 499, devtest: 506 instances)
Creating dataloaders for tel_Telu (dev: 499, devtest: 506 instances)
Creating dataloaders for tam_Taml (dev: 499, devtest: 506 instances)
Creating dataloaders for quy_Latn (dev: 499, devtest: 506 instances)


In [16]:
# let's take a look at a tokenized sample
def get_batch_containing_sample(data_loader, sample_id, batch_size):
    for batch_idx, batch in enumerate(data_loader):
        if sample_id < (batch_idx + 1) * batch_size and sample_id >= batch_idx * batch_size:
            return batch
    return None

def get_sample_from_batch(sample_id: int, lang: str, split: str, data_type: str, batch_size: int = BATCH_SIZE, per_batch_padding: bool = PER_BATCH_PADDING):
    if data_type == "raw":
        # Return the raw sample with the normal `get_sample` function
        sample = get_sample(sample_id, lang, split, "raw")
        return sample
    elif data_type == "tokenized" and not per_batch_padding:
        # Return the tokenized sample with the normal `get_sample` function
        sample = get_sample(sample_id, lang, split, "tokenized")
        return sample
    elif data_type == "tokenized" and per_batch_padding:
        # We fetch the batch first that contains the wanted `sample_id`
        data_loader = dataset_per_lang[lang]['dataloader'][split]
        batch = get_batch_containing_sample(data_loader, sample_id, batch_size)
        if batch is not None:
            # Adjust indexing based on `batch` and `sample_id`
            index_within_batch = sample_id % batch_size
            tokenized_sample = {
                'input_ids': batch['input_ids'][index_within_batch],
                'attention_mask': batch['attention_mask'][index_within_batch]
            }
            return tokenized_sample
    else:
        return None

# Sample ID and language
LOOKAT_SAMPLE_ID = 17
EX_DATASET_LANG = "eng_Latn"

# get raw and tokenized sample
raw_sample = get_sample_from_batch(LOOKAT_SAMPLE_ID, EX_DATASET_LANG, "dev", "raw")
tokenized_sample = get_sample_from_batch(LOOKAT_SAMPLE_ID, EX_DATASET_LANG, "dev", "tokenized")

print(f"Viewing {LOOKAT_SAMPLE_ID}-th sample from {EX_DATASET_LANG}:")
print("\tRaw sample:")
pprint_tab(raw_sample, indent="\t\t")
print("\n\tTokenized sample:")
pprint_tab(tokenized_sample, indent="\t\t")

EX_DATASET_LANG = "spa_Latn"

raw_sample = get_sample_from_batch(LOOKAT_SAMPLE_ID, EX_DATASET_LANG, "dev", "raw")
tokenized_sample = get_sample_from_batch(LOOKAT_SAMPLE_ID, EX_DATASET_LANG, "dev", "tokenized")

print(f"\nViewing {LOOKAT_SAMPLE_ID}-th sample from {EX_DATASET_LANG}:")
print("\tRaw sample:")
pprint_tab(raw_sample, indent="\t\t")
print("\n\tTokenized sample:")
pprint_tab(tokenized_sample, indent="\t\t")

Viewing 17-th sample from eng_Latn:
	Raw sample:
		{'URL': 'https://en.wikinews.org/wiki/Investigation_of_Deutsche_Bank_headquarters_spills_into_second_day',
		 'domain': 'wikinews',
		 'has_hyperlink': 0,
		 'has_image': 0,
		 'id': 18,
		 'sentence': 'British newspaper The Guardian suggested Deutsche Bank '
		             'controlled roughly a third of the 1200 shell companies used to '
		             'accomplish this.',
		 'topic': 'crime'}

	Tokenized sample:
		{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
		        1, 1, 1, 1, 1, 1]),
		 'input_ids': tensor([     2,   4934, 133365,     67,  70145,    235,     11,  32607,   1589,
		          4572, 148652,     48,   3990,   3847, 116845,      7,      4,     11,
		        116370,      8,   7285,  22176,   4128,   6763,      8,  19362,    196,
		         86614,  56115,      5])}

Viewing 17-th sample from spa_Latn:
	Raw sample:
		{'URL': 'https://en.wikinews.org/wiki/Investigation_o

## Load model

In [17]:
# load pre-trained model from the huggingface hub
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir="../cache/models")

# specify device on model and put the model into evaluation mode
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
if torch.cuda.is_available():
    model = model.cuda()
model = model.eval()
print(f"Using device: {device}")

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Using device: cpu


In [18]:
# define the cross-entropy loss function
def cross_entropy_loss(logits: torch.Tensor, labels: torch.Tensor, attention_mask: torch.Tensor) -> float:
    # logits are of shape: (batch_size, sequence_length, vocab_size)
    # convert logits to probabilities in the last dimension
    # that is, probs[:, i] = e^{probs[:, i]} / sum(e^{probs[:, j]})
    probs = torch.nn.functional.softmax(logits, dim=-1) # shape: [1, 84, 256008]

    # use `torch.gather` to pick out the probabilities of the actual tokens of the sentence given by the labels
    # these probabilities are saved in the last dimension, so dim=-1
    # we need to unsqueeze labels to match the dimensions of `probs`, and then squeeze back the result
    # see: https://stackoverflow.com/questions/50999977/what-does-the-gather-function-do-in-pytorch-in-layman-terms
    index = labels.squeeze(1).unsqueeze(-1) # shape: (batch_size, sequence_length, 1), e.g, [2, 1, 73] -> [2, 73, 1]
    actual_probs = torch.gather(probs, dim=-1, index=index).squeeze(-1)

    # compute the negative log-likelihood, adding a small offset to prevent log(0)
    nll = -torch.log(actual_probs + 1e-9)

    # apply the attention mask to ignore padding positions
    attention_mask = attention_mask.to(dtype=nll.dtype)
    nll_masked = nll * attention_mask

    # sum up the negative log-likelihood of each token (taking into account the mask) and dividing by the number of
    # actual tokens in the sentence to get the Expected Value
    ce_loss = torch.sum(nll_masked) / torch.sum(attention_mask)
    return ce_loss.item()

In [39]:
# test cross-entropy loss on a sample
USE_MANUAL_CROSS_ENTROPY_LOSS_FN = False # Set to True if `cross_entropy_loss` is to be used

inputs = tokenized_sample["input_ids"].unsqueeze(0).to(device) # Add batch dimension, so that dimension match the expected model input
labels = tokenized_sample["input_ids"].unsqueeze(0).to(device)
attention_mask = tokenized_sample["attention_mask"].unsqueeze(0).to(device)

# torch.inference_mode() is now preferred over torch.no_grad().
# See: https://discuss.pytorch.org/t/pytorch-torch-no-grad-vs-torch-inference-mode/134099/2?u=timgianitsos
with torch.inference_mode():
    outputs = model(
        inputs,
        labels=labels,
        attention_mask=attention_mask,
    )
    if USE_MANUAL_CROSS_ENTROPY_LOSS_FN:
        loss = cross_entropy_loss(outputs.logits, labels, attention_mask)
    else:
        loss = outputs.loss.item()

print(f"Outputs shape: {outputs.logits.shape}") # (batch_size, sequence_length, vocab_size)
print(f"Cross-entropy loss of the sample: {loss}")

Outputs shape: torch.Size([1, 54, 256008])
Cross-entropy loss of the sample: 4.964214324951172


In [37]:
# store per-batch losses for each split in each language
losses = {lang: {} for lang in LANGUAGES}

# Frees unused memory so it can be used by other tensors
torch.cuda.empty_cache()
try:
    # delete variables if they exist
    del inputs, labels, outputs
except NameError:
    pass

# TODO: Revise this to make sure it uses the cross-entropy loss
# iterate over the dataset for each language and compute the cross-entropy loss per batch
for language in dataset_per_lang:
    print(f"Computing losses for {language} (", end="")
    for j, split in enumerate(dataset_per_lang[language]["dataset"]):
        print(f"{split}: ", end="")
        dataloader = dataset_per_lang[language]["dataloader"][split]
        losses[language][split] = []
        for i, batch in enumerate(dataloader):
            inputs = batch["input_ids"].to(device)
            labels = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # torch.inference_mode() is now preferred over torch.no_grad().
            # See: https://discuss.pytorch.org/t/pytorch-torch-no-grad-vs-torch-inference-mode/134099/2?u=timgianitsos
            with torch.inference_mode():
                outputs = model(inputs, labels=labels, attention_mask=attention_mask)
                if USE_MANUAL_CROSS_ENTROPY_LOSS_FN:
                    loss = cross_entropy_loss(outputs.logits, labels, attention_mask)
                else:
                    loss = outputs.loss.item()

            losses[language][split].append(loss)

            # Explicitly delete tensors to free up GPU memory
            del inputs, labels, attention_mask, outputs

        print(f"{np.mean(losses[language][split])}", end="")
        if j < len(dataset_per_lang[language]["dataset"]) - 1:
            print(", ", end="")

    print(" per-batch mean)")

    # After processing each language, try to free up memory explicitly
    torch.cuda.empty_cache()  # Frees unused memory so it can be used by other tensors

Computing losses for eng_Latn (dev: 9.170358657836914
7.607446193695068
9.788981437683105
6.797296047210693
9.210999488830566
9.072386741638184
9.196784973144531
14.03604507446289


KeyboardInterrupt: 

In [ ]:
# save the losses to a json file
if not os.path.exists("../cache/losses"):
    os.makedirs("../cache/losses")

with open(f"../cache/losses/losses_batchsize_{BATCH_SIZE}_perbatchpadding_{PER_BATCH_PADDING}.json", "w") as f:
    json.dump(losses, f, sort_keys=True, indent=4)

## Visualize loss per language

In [ ]:
# load the variable `losses` from the json file if it doesn't exist
try:
    # check if `losses` is defined and has data
    if not 'losses' in locals() or not losses:
        raise NameError  # Trigger the exception if not defined or empty
except NameError:
    # if `losses` is not defined or is empty, load it from the JSON file, if it exists
    # if the JSON file does not exists, then nothing will be loaded, therefore the
    # `Load Model` cells must be ran
    if os.path.exists(f"../cache/losses/losses_{BATCH_SIZE}.json"):
        with open(f"../cache/losses/losses_{BATCH_SIZE}.json", "r") as f:
            losses = json.load(f)

In [ ]:
SPLIT_TO_CHECK = "devtest"

# create a figure
fig, axes = plt.subplots(figsize=(8, 5))

# create a bar plot for each language
x = np.arange(len(LANGUAGES))
y = [np.mean(losses[_][SPLIT_TO_CHECK]) for _ in LANGUAGES]

axes.plot(x, y)

fig.tight_layout()

# format plot
axes.set_xlabel("Language") # x-axis label
axes.set_xticks(range(len(LANGUAGES))) # x-axis ticks
axes.set_xticklabels(losses.keys()) # x-axis tick labels
axes.set_ylabel("Loss") # y-axis label
axes.set_title(MODEL_NAME); # title
axes.grid(True, which='major', color='k', linestyle='-', alpha=0.2)
axes.grid(True, which='minor', color='k', linestyle='--', alpha=0.1)
axes.minorticks_on()

## Comparing XGLM to GPT2

Your next task is to re-run the analysis above, but using `gpt2` as the pre-trained language model. For this exercise, focus on your native language, unless it's English or isn't covered by flores. In that case, pick another language that you can read well.

Compare the language modeling loss of XGLM and GPT2. What do you observe? Investigate the differences in tokenization for XGLM and GPT2. What do you observe? How can the good (or bad) performance of GPT2 be explained?

In [ ]:
# TODO: your code goes here